In [20]:
from guardrails import Guard
import google.generativeai as genai
import json
from typing import Optional

## Loading the API key for Gemini

In [21]:
def load_api_key():
    with open('config.json') as config_file:
        config = json.load(config_file)
        return config['apiKey']

In [22]:
# configuring the gemini model with the api key
genai.configure(api_key=load_api_key())

# Configuring the Gemini Model

In [23]:
# configurations for the model
gen_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

In [24]:
# selecting the AI model for the response
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    generation_config=gen_config,
)

In [25]:
# for now there is no history for the chat
chat_session = model.start_chat(
    history=[
    ]
)

# Defining a custom LLM

In [28]:
def my_llm_api(
    prompt: Optional[str] = None,
    instruction: Optional[str] = None,
    msg_history: Optional[list[dict]] = None,
    **kwargs
) -> str:
    """Custom LLM API wrapper.

    At least one of prompt, instruction or msg_history should be provided.

    Args:
        prompt (str): The prompt to be passed to the LLM API
        instruction (str): The instruction to be passed to the LLM API
        msg_history (list[dict]): The message history to be passed to the LLM API
        **kwargs: Any additional arguments to be passed to the LLM API

    Returns:
        str: The output of the LLM API
    """

    # Call your LLM API here
    llm_output = chat_session.send_message(prompt).text

    return llm_output

# Using the RAIL spec file to generate response

In [29]:
guard = Guard.from_rail('./guardrail.xml')
_, validated_output, *rest = guard(
    my_llm_api,
)

In [30]:
# print(guard)
print(validated_output)

{'fees': [{'index': 1, 'name': 'transaction fee', 'explanation': 'Fee for processing transactions', 'value': 1.5}, {'index': 2, 'name': 'monthly maintenance fee', 'explanation': 'Monthly fee for account maintenance', 'value': 5.0}, {'index': 3, 'name': 'overdraft fee', 'explanation': 'Fee for exceeding account balance', 'value': 35.0}], 'interest_rates': 'null'}
